In [ ]:
from __future__ import print_function

from simtk.openmm import app
import simtk.openmm as mm
from simtk import unit
import sys
import mbpol

In [ ]:
nonbondedMethod=app.PME
expected_energy = -2270.88890
pdb = app.PDBFile("pdb_files/water256_integration_test.pdb")
forcefield = app.ForceField("../mbpol.xml")

In [ ]:
#forcefield._forces.pop(1)
forcefield._forces[2:3]

In [ ]:
      
nonbondedCutoff = 0.9*unit.nanometer
        
if (nonbondedMethod == app.PME):
    boxDimension = 19.3996888399961804/10.
    boxsize = [boxDimension, boxDimension, boxDimension]
    pdb.topology.setUnitCellDimensions( boxsize )
            
system = forcefield.createSystem(pdb.topology, nonbondedMethod=nonbondedMethod, nonbondedCutoff=nonbondedCutoff)

In [ ]:
for  i in range(system.getNumForces()):
    system.getForce(i).setForceGroup(i)
    print (system.getForce(i))
forceLabels = ["Electrostatics", "TwoBody", "ThreeBody", "CMMotionRemover", "CustomDispersion"]
#forceLabels = ["Electrostatics", "OneBody", "TwoBody", "ThreeBody", "CMMotionRemover", "CustomDispersion"]

In [ ]:
            
integrator = mm.LangevinIntegrator(0.0, 0.1, 0.01)
platform = mm.Platform.getPlatformByName('Reference')
simulation = app.Simulation(pdb.topology, system, integrator, platform)
simulation.context.setPositions(pdb.positions)
simulation.context.computeVirtualSites()
state = simulation.context.getState(getForces=True, getEnergy=True, getPositions=True)
potential_energy = state.getPotentialEnergy()
potential_energy.in_units_of(unit.kilocalorie_per_mole)

In [ ]:
for  i in range(system.getNumForces()):
    state = simulation.context.getState(getEnergy=True, groups=pow(2,i))
    print("Energy {}: {}".format(forceLabels[i], state.getPotentialEnergy().in_units_of(unit.kilocalorie_per_mole)))

In [ ]:
        
print(potential_energy.in_units_of(unit.kilocalorie_per_mole)._value)
        
        
print(abs(potential_energy.in_units_of(unit.kilocalorie_per_mole)._value - expected_energy) < 1)